In [1]:
from cwdb import (
    CWComplex,
    Cell,
    to_lang_representation
)
from cwdb.rules import apply_rule

In [2]:
from cwdb.ilogic import get_truth_value

In [3]:
import numpy as np

In [4]:
c = CWComplex()

robin = c.create_cell("robin")
bird = c.create_cell("bird")
animal = c.create_cell("animal")
water = c.create_cell("water")
liquid = c.create_cell("liquid")
vertebrate = c.create_cell("vertebrate")

animal_have_wings = c.create_cell("animal_have_wings")
animal_flying = c.create_cell("animal_flying")

is_1 = c.create_cell("is", [c["robin"], bird])
c.create_cell("is", [robin, animal_have_wings])
c.create_cell("is", [animal_have_wings, animal_flying])
c.create_cell("is", [animal_flying, animal])
is_2 = c.create_cell("is", [bird, vertebrate])
c.create_cell("is", [vertebrate, animal])
c.create_cell("is", [water, liquid])

Cell("is", embedding=[], dimension=1, boundary=["water", "liquid"])

In [5]:
# Difficulty: run confidence revision based on part of the database
# Two rules to resolve "TruthValue" inconsistency:
#     1. revision rule
#     2. choice rule

In [6]:
c.build_coboundary()

In [7]:
for e in c.layers[1]:
    x, y  = e.boundary[:2]
    e.data.embedding = np.array(get_truth_value(x, y))

In [8]:
is_2

Cell("is", embedding=[1.         0.66666667], dimension=1, boundary=["bird", "vertebrate"])

In [9]:
is_2_dup = c.create_cell("is", [bird, vertebrate])
is_2_dup.data.embedding = np.array([0.5, 0.8])

In [10]:
class QueryError(RuntimeError):
    def __init__(self, message, offending):
        super().__init__(message)
        self.offending = offending

In [11]:
def query(label:str, boundary):
    x, y  = boundary[:2]
    result = None
    for e in x.coboundary:
        if e.dimension == 1 and e.label == label and e.boundary[1] is y:
            if result is None:
                result = e
                continue
            else:
                raise QueryError("Uncertainty detected", [result, e])
    if result is None:
        return np.array([0.5, 0])
    return result.embedding

In [12]:
c.build_coboundary()

In [13]:
#query("is", [bird, vertebrate])

In [14]:
query("is", [robin, bird])

array([1.        , 0.66666667])

In [15]:
query("is", [robin, water])

array([0.5, 0. ])

In [16]:
def revision_rule(e1:Cell, e2:Cell) -> Cell:
    assert e1.label == e2.label
    assert e1.boundary == e2.boundary
    f1, c1 = e1.embedding
    f2, c2 = e2.embedding
    f3 = (f1 * c1 * (1 - c2) + f2 * c2 * (1 - c1) ) / (c1 * (1-c2) + c2 * (1-c1) )
    c3 = (c1 * (1-c2) + c2 * (1-c1) ) / ( (c1 * (1-c2) + c2 * (1-c1) ) + (1-c1)*(1-c2) )
    
    result = c.create_cell(e1.label, e1.boundary)
    result.data.embedding = np.array([f3, c3])
    e1.data.deleted = True
    e2.data.deleted = True
    return result
    

In [17]:
def choice_rule(e1:Cell, e2:Cell) -> Cell:
    assert e1.label == e2.label
    assert e1.boundary == e2.boundary
    f1, c1 = e1.embedding
    f2, c2 = e2.embedding
    if c1 > c2:
        e1, e2 = e2, e1
    e1.data.deleted = True
    return e2


In [27]:
def random_rule(e1:Cell, e2:Cell) -> Cell:
    assert e1.label == e2.label
    assert e1.boundary == e2.boundary
    if np.random.random() > 0.5:
        rule = choice_rule
    else:
        rule = revision_rule
    return rule(e1, e2)

In [19]:
print(to_lang_representation(c))

robin: 'robin'
bird: 'bird'
animal: 'animal'
water: 'water'
liquid: 'liquid'
vertebrate: 'vertebrate'
animal_have_wings: 'animal_have_wings'
animal_flying: 'animal_flying'
is: robin->bird: 'is'
is_1: robin->animal_have_wings: 'is'
is_2: animal_have_wings->animal_flying: 'is'
is_3: animal_flying->animal: 'is'
is_4: bird->vertebrate: 'is'
is_5: vertebrate->animal: 'is'
is_6: water->liquid: 'is'
is_7: bird->vertebrate: 'is'



In [20]:
#revision_rule(is_2, is_2_dup)

In [21]:
#choice_rule(is_2, is_2_dup)

In [22]:
print(to_lang_representation(c))

robin: 'robin'
bird: 'bird'
animal: 'animal'
water: 'water'
liquid: 'liquid'
vertebrate: 'vertebrate'
animal_have_wings: 'animal_have_wings'
animal_flying: 'animal_flying'
is: robin->bird: 'is'
is_1: robin->animal_have_wings: 'is'
is_2: animal_have_wings->animal_flying: 'is'
is_3: animal_flying->animal: 'is'
is_4: bird->vertebrate: 'is'
is_5: vertebrate->animal: 'is'
is_6: water->liquid: 'is'
is_7: bird->vertebrate: 'is'



In [23]:
def mega_query(label:str, boundary):
    while True:
        try:
            return query(label, boundary)
        except QueryError as e:
            random_rule(*e.offending)

In [24]:
print(to_lang_representation(c))

robin: 'robin'
bird: 'bird'
animal: 'animal'
water: 'water'
liquid: 'liquid'
vertebrate: 'vertebrate'
animal_have_wings: 'animal_have_wings'
animal_flying: 'animal_flying'
is: robin->bird: 'is'
is_1: robin->animal_have_wings: 'is'
is_2: animal_have_wings->animal_flying: 'is'
is_3: animal_flying->animal: 'is'
is_4: bird->vertebrate: 'is'
is_5: vertebrate->animal: 'is'
is_6: water->liquid: 'is'
is_7: bird->vertebrate: 'is'



In [25]:
mega_query("is", [bird, vertebrate])

array([0.5, 0.8])

In [26]:
print(to_lang_representation(c))

robin: 'robin'
bird: 'bird'
animal: 'animal'
water: 'water'
liquid: 'liquid'
vertebrate: 'vertebrate'
animal_have_wings: 'animal_have_wings'
animal_flying: 'animal_flying'
is: robin->bird: 'is'
is_1: robin->animal_have_wings: 'is'
is_2: animal_have_wings->animal_flying: 'is'
is_3: animal_flying->animal: 'is'
DELETED_is: bird->vertebrate: '[DELETED] is'
is_4: vertebrate->animal: 'is'
is_5: water->liquid: 'is'
is_6: bird->vertebrate: 'is'



In [30]:
def unambiguity_resolution_rule(e1:Cell, e2:Cell) -> Cell:
    assert e1.label == e2.label
    assert e1.boundary == e2.boundary
    # e2.data.deleted = True
    result = e1
    # assert (e1.data.deleted xor e2.data.deleted) or (new_result and e1.data.deleted and e2.data.deleted)
    return e1

In [ ]:
# Create Task

is_type = c.create_cell("is_type")
task = c.create_cell("task")
arguments = c.create_cell("arguments")
